In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large

import src.graphics as graphics

In [2]:
VERSION = 1
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV3"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [3]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [4]:
OPTIMIZERS = {"Adam": tf.keras.optimizers.legacy.Adam()}

In [5]:
LOSS = {"BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy()}

In [6]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

2025-02-28 19:38:34.793295: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-02-28 19:38:34.793339: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-28 19:38:34.793350: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-28 19:38:34.793397: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-28 19:38:34.793429: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
BATCH_SIZES = [16, 32, 64, 128]

In [8]:
SUBVERSION = 0
for BATCH_SIZE in BATCH_SIZES:
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.MobileNetV3Large(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS["Adam"],
        loss=LOSS["BinaryCrossentropy"],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # y_pred_prob = y_pred_prob.flatten()
    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    # Crear el gráfico correctamente
    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
12683000/12683000 [==============================] - 1s 0us/step
Epoch 1/10


2025-02-28 19:38:39.498852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1120/1120 [==============================] - ETA: 0s - loss: 0.2213 - accuracy: 0.9045 - precision: 0.9308 - recall: 0.9407 - auc: 0.9611

2025-02-28 19:40:16.625589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1120/1120 [==============================] - 120s 104ms/step - loss: 0.2213 - accuracy: 0.9045 - precision: 0.9308 - recall: 0.9407 - auc: 0.9611 - val_loss: 0.1912 - val_accuracy: 0.9253 - val_precision: 0.9608 - val_recall: 0.9249 - val_auc: 0.9750
Epoch 2/10
1120/1120 [==============================] - 137s 122ms/step - loss: 0.1685 - accuracy: 0.9317 - precision: 0.9551 - recall: 0.9523 - auc: 0.9774 - val_loss: 0.1806 - val_accuracy: 0.9260 - val_precision: 0.9248 - val_recall: 0.9669 - val_auc: 0.9808
Epoch 3/10
1120/1120 [==============================] - 141s 126ms/step - loss: 0.1561 - accuracy: 0.9348 - precision: 0.9573 - recall: 0.9543 - auc: 0.9808 - val_loss: 0.1645 - val_accuracy: 0.9349 - val_precision: 0.9571 - val_recall: 0.9439 - val_auc: 0.9813
Epoch 4/10
1120/1120 [==============================] - 153s 137ms/step - loss: 0.1495 - accuracy: 0.9384 - precision: 0.9604 - recall: 0.9560 - auc: 0.9823 - val_loss: 0.1621 - val_accuracy: 0.9370 - val_precision: 0.9524 - 

...

140/140 [==============================] - 133s 922ms/step - loss: 0.2419 - accuracy: 0.9020 - precision: 0.9293 - recall: 0.9358 - auc: 0.9614 - val_loss: 0.2029 - val_accuracy: 0.9260 - val_precision: 0.9542 - val_recall: 0.9330 - val_auc: 0.9723
Epoch 2/10
140/140 [==============================] - 122s 872ms/step - loss: 0.1753 - accuracy: 0.9283 - precision: 0.9524 - recall: 0.9505 - auc: 0.9761 - val_loss: 0.1863 - val_accuracy: 0.9305 - val_precision: 0.9376 - val_recall: 0.9587 - val_auc: 0.9780
Epoch 3/10
140/140 [==============================] - 126s 899ms/step - loss: 0.1636 - accuracy: 0.9337 - precision: 0.9549 - recall: 0.9553 - auc: 0.9792 - val_loss: 0.1767 - val_accuracy: 0.9365 - val_precision: 0.9582 - val_recall: 0.9453 - val_auc: 0.9782
Epoch 4/10
140/140 [==============================] - 124s 888ms/step - loss: 0.1566 - accuracy: 0.9353 - precision: 0.9567 - recall: 0.9557 - auc: 0.9811 - val_loss: 0.1712 - val_accuracy: 0.9349 - val_precision: 0.9581 - val_reca

INFO:tensorflow:Assets written to: /Users/carlos/WORKSPACE/MegaClassifier/models/MegaClassifier_a_MobileNetV3/v1/v1.3/assets


34/34 [==============================] - 28s 785ms/step - loss: 0.1505 - accuracy: 0.9358 - precision: 0.9598 - recall: 0.9425 - auc: 0.9847


2025-02-28 21:11:59.100801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


34/34 [==============================] - 26s 749ms/step



